# Contains code for deploying the trained model

In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")


In [2]:
# bucket = 'project-completion-udacity'
# dataset_name = 'nsfw_dataset'


# s3_output_location = 's3://project-completion-udacity/nsfw_dataset/output'

# image_classifier = sagemaker.estimator.Estimator(
#     training_image,
#     role, 
#     train_instance_count = 1, 
#     train_instance_type = 'ml.p2.xlarge',
#     output_path=s3_output_location,
#     sagemaker_session=sess
# )

# the only change you need to make is to change job_name with the name of your own training job.

In [3]:


job_name = 'IC-nsfw-dataset-1568090782'
import boto3
from time import gmtime, strftime
import time
sage = boto3.Session().client(service_name='sagemaker') 
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name="DEMO-full-image-classification-model" + timestamp
print(model_name)
info = sage.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

DEMO-full-image-classification-model-2019-09-11-06-17-19
s3://project-completion-udacity/nsfw_dataset/output/IC-nsfw-dataset-1568090782/output/model.tar.gz
arn:aws:sagemaker:us-east-2:733184320490:model/demo-full-image-classification-model-2019-09-11-06-17-19


<h2> For endpoint configuration

In [ ]:
# %%time
# # Deploying a model to an endpoint takes a few minutes to complete

# deployed_endpoint = image_classifier.deploy(
#     initial_instance_count = 1,
#     instance_type = 'ml.t2.medium'
# )

from time import gmtime, strftime

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = 'my-endpoint' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.t2.medium',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

## code for lauanching the endpoint.

In [5]:
endpoint_name = 'myprojectcapstone'
print('Endpoint name: {}'.format('myprojectcapstone'))

import sagemaker as sm

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sess.sagemaker_client.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: myprojectcapstone
EndpointArn = arn:aws:sagemaker:us-east-2:733184320490:endpoint/myprojectcapstone


In [6]:
endpoint_dec = sess.wait_for_endpoint(endpoint_name)

------------------------------------------------------------------------------------------------------------!

## Testing the deployed endpoint

In [7]:
import json
import numpy as np
import os

def classify_deployed(file_name, classes , endpoint_name):
    payload = None
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)

    response = sess.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'application/x-image',
                                                Body = payload)
        
    #print(response)
    result = response['Body'].read()
    result = json.loads(result)
    print(result)
    best_prob_index = np.argmax( np.array( result ) )
    print(best_prob_index)
    return (classes[best_prob_index], result[best_prob_index])

In [8]:
file_name = 'flowes/test/daisy/10555826524_423eb8bf71_n.jpg'
classes = ['animated' , 'nude' ,'porn' , 'safe_for_work' , 'semi_nude']

class_predicted , class_score = classify_deployed(file_name, classes , endpoint_name)
print(class_predicted , class_score)


[0.0002461728290654719, 0.0017670468660071492, 0.00032288278453052044, 0.9976606369018555, 3.218596020815312e-06]
3
safe_for_work 0.9976606369018555


## Deleting the endpoint.

In [9]:
sess.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': '75b459e9-b20b-466b-aab3-17afe9b8fa29',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '75b459e9-b20b-466b-aab3-17afe9b8fa29',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 11 Sep 2019 06:30:04 GMT'},
  'RetryAttempts': 0}}